# Classification with KNN, Trees and Gaussian Naive Bayes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Load and split the data from the Unsupervise Learning Dataset (Lab 5, Dry Bean Dataset):

In [2]:
FFILE = './Dry_Bean_Dataset.xlsx'
if os.path.isfile(FFILE):
    print("File already exists")
    if os.access(FFILE, os.R_OK):
        print ("File is readable")
    else:
        print ("File is not readable, removing it and downloading again")
        !rm FFILE
        !wget "https://raw.github.com/alexdepremia/ML_IADA_UTs/main/Lab5/Dry_Bean_Dataset.xlsx"
else:
    print("Either the file is missing or not readable, download it")
    !wget "https://raw.github.com/alexdepremia/ML_IADA_UTs/main/Lab5/Dry_Bean_Dataset.xlsx"

File already exists
File is readable


In [3]:
# Load the data
data = pd.read_excel('./Dry_Bean_Dataset.xlsx', engine='openpyxl')
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


Divide features and label. Split the data in train and test set and **after that** normalize them:

In [4]:
data = data.sample(frac=1,random_state=0).reset_index(drop=True) # random shuffle
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,37277,710.193,264.789840,179.808422,1.472622,0.734082,37684,217.859015,0.802692,0.989200,0.928748,0.822762,0.007103,0.002008,0.676937,0.996873,DERMASON
1,28942,638.821,239.861192,154.004371,1.557496,0.766658,29368,191.963796,0.786126,0.985494,0.891210,0.800312,0.008288,0.002097,0.640499,0.997575,DERMASON
2,38290,719.888,270.446510,180.508066,1.498252,0.744659,38605,220.799326,0.759903,0.991840,0.928465,0.816425,0.007063,0.001936,0.666550,0.998660,DERMASON
3,37641,742.538,284.313737,169.740814,1.674987,0.802227,38112,218.920099,0.744187,0.987642,0.857894,0.769995,0.007553,0.001638,0.592892,0.993087,SIRA
4,50172,828.968,316.453571,202.268818,1.564520,0.769062,50547,252.746858,0.688240,0.992581,0.917478,0.798685,0.006307,0.001583,0.637898,0.998005,SEKER


In [5]:
data.shape

(13611, 17)

In [6]:
data['Class'].unique()

array(['DERMASON', 'SIRA', 'SEKER', 'CALI', 'BOMBAY', 'HOROZ', 'BARBUNYA'],
      dtype=object)

In [7]:
train_data = data.iloc[:10000,:]
test_data = data.iloc[10000:,:]

In [8]:
print(train_data.shape)
print(test_data.shape)

(10000, 17)
(3611, 17)


In [9]:
# normalize train and test dataset
from sklearn import preprocessing

# Extract the class labels from the training dataset
label_train = train_data['Class']
# Remove the class labels from the training dataset
train_data = train_data.drop('Class', axis=1)
# Save the column names for later use
columns_name = train_data.columns

# Initialize a StandardScaler and fit it to the training data
train_scaler = preprocessing.StandardScaler().fit(train_data)
# Transform the training data using the scaler
train_data = train_scaler.transform(train_data)
# Create a DataFrame with the scaled training data and restore column names
train_data = pd.DataFrame(train_data, columns=columns_name)
# Add the class labels back to the scaled training dataset
train_data['Class'] = label_train

# Extract the class labels from the test dataset
label_test = test_data['Class']
# Remove the class labels from the test dataset
test_data = test_data.drop('Class', axis=1)
# Initialize a StandardScaler and fit it to the test data
test_scaler = preprocessing.StandardScaler().fit(test_data)
# Transform the test data using the scaler
test_data = test_scaler.transform(test_data)
# Create a DataFrame with the scaled test data and restore column names
test_data = pd.DataFrame(test_data, columns=columns_name)
# Add the class labels back to the scaled test dataset
test_data['Class'] = label_test.values # .values added to prevent nans to appear

In [10]:
train_data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,-0.534137,-0.673988,-0.642784,-0.494897,-0.448854,-0.185185,-0.536571,-0.590932,1.083156,0.441530,0.932328,0.372606,0.473404,0.490160,0.338721,0.416048,DERMASON
1,-0.819126,-1.008115,-0.934618,-1.070089,-0.104138,0.170622,-0.816632,-1.029793,0.746572,-0.348554,0.302277,0.007584,1.526541,0.640814,-0.030790,0.576165,DERMASON
2,-0.499500,-0.628601,-0.576563,-0.479301,-0.344759,-0.069652,-0.505554,-0.541101,0.213791,1.004632,0.927566,0.269572,0.437664,0.368508,0.233385,0.823378,DERMASON
3,-0.521691,-0.522565,-0.414222,-0.719312,0.373055,0.559127,-0.522157,-0.572949,-0.105517,0.109317,-0.256904,-0.485355,0.873546,-0.133675,-0.513569,-0.447017,SIRA
4,-0.093232,-0.117944,-0.037969,0.005762,-0.075610,0.196889,-0.103380,0.000331,-1.242245,1.162580,0.743167,-0.018863,-0.234347,-0.225790,-0.057166,0.674089,SEKER


In [11]:
test_data['Class'].unique()

array(['DERMASON', 'SEKER', 'SIRA', 'BARBUNYA', 'HOROZ', 'BOMBAY', 'CALI'],
      dtype=object)

**Before feeding the data into the following algorithms, try to perform PCA, varying the number of PCs, and check what changes**

## K-Nearest Neighbors Classification

Implement the KNN algorithm for classification.

In [12]:
from scipy.spatial.distance import euclidean, minkowski
from tqdm import tqdm

def distance(point_one, point_two, dist, p):
    """
    Calculate the Euclidean distance between two points.

    Parameters
    ----------
    point_one : array-like
        Coordinates of the first point.
    point_two : array-like
        Coordinates of the second point.
    dist: str
        Allow to choose between Euclidean or Minkowski distance.
    p: int
        Order of the norm, only used with Minkowski distance.

    Returns
    -------
    float
        Euclidean or Minkowski distance between the two points.
    """
    if dist == 'euclidean':
        return euclidean(point_one, point_two)
    else:
        return minkowski(point_one, point_two, p=p)


def get_neighbors(train_set, test_point, label_col, n_neighbors, dist, p):
    """
    Get the nearest neighbors of a test point in the training set.

    Parameters
    ----------
    train_set : array-like
        The training set containing data points.
    test_point : array-like
        The test point for which neighbors are to be found.
    label_col : array-like
        The labels corresponding to the training set.
    n_neighbors : int
        The number of neighbors to retrieve.
    dist: str
        Allow to choose between Euclidean or Minkowski distance.
    p: int
        Order of the norm, only used with Minkowski distance.

    Returns
    -------
    ordered_train : array-like
        The nearest neighbors in the training set.
    ordered_label : array-like
        The corresponding labels of the nearest neighbors.
    """
    # Calculate distances between the test point and all points in the training set
    dist = np.array([distance(train_point, test_point, dist, p) for train_point in train_set])
    # Get indices that would sort the distances in ascending order
    idx_dist = dist.argsort()
    # Order the training set and labels based on the sorted distances
    ordered_train = train_set[idx_dist, :]
    ordered_label = label_col[idx_dist]
    # Return the top n_neighbors neighbors and their labels
    return ordered_train[:n_neighbors], ordered_label[:n_neighbors]

def predict(train_set, test_point, labels, n_neighbors, dist, p):
    """
    Predict the label of a test point using k-nearest neighbors.

    Parameters
    ----------
    train_set : array-like
        The training set containing data points.
    test_point : array-like
        The test point for which the label is to be predicted.
    labels : array-like
        The labels corresponding to the training set.
    n_neighbors : int
        The number of neighbors to consider for the prediction.
    dist: str
        Allow to choose between Euclidean or Minkowski distance.
    p: int
        Order of the norm, only used with Minkowski distance.

    Returns
    -------
    predicted_label : array-like
        The predicted label for the test point.
    """
    # Get the nearest neighbors and their labels
    neigh, neigh_label = get_neighbors(train_set, test_point, labels, n_neighbors, dist, p)
    # Count occurrences of each label among the neighbors
    values, counts = np.unique(neigh_label, return_counts=True)
    # Find the label with the highest count (majority class)
    idx = np.argmax(counts)
    # Return the predicted label
    return values[idx]

def evaluate(train_set, test_set, label, n_neighbors=2, dist='Euclidean', p=2):
    """
    Evaluate the accuracy of k-nearest neighbors algorithm on a test set.

    Parameters
    ----------
    train_set : DataFrame
        The training dataset.
    test_set : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.
    n_neighbors : int, optional
        The number of neighbors to consider for the prediction. Default is 2.
    dist: str
        Allow to choose between Euclidean or Minkowski distance.
    p: int
        Order of the norm, only used with Minkowski distance.

    Returns
    -------
    accuracy : float
        The accuracy of the k-nearest neighbors algorithm on the test set.
    """
    # Initialize counters for correct and incorrect predictions
    correct_predict = 0
    wrong_predict = 0
    # Extract labels and features from the training and test sets
    
    train_labels = train_set[label].values
    train_set = train_set.drop(label, axis=1)
    
    test_labels = test_set[label].values
    test_set = test_set.drop(label, axis=1)
    # Iterate through each row in the test dataset
    for index in tqdm(range(len(test_set.index)), desc="Evaluating KNN"):
        # Predict the class label for the current test row
        result = predict(train_set.values, test_set.iloc[index].values, train_labels, n_neighbors, dist, p)
        # Check if the predicted value matches the actual value
        if result == test_labels[index]:
            # Increase the correct prediction count
            correct_predict += 1
        else:
            # Increase the incorrect prediction count
            wrong_predict += 1

    # Calculate and return the accuracy
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy


In [13]:
knn_accuracy = evaluate(train_data, test_data, 'Class', n_neighbors=5)

Evaluating KNN: 100%|██████████| 3611/3611 [04:34<00:00, 13.15it/s]


In [14]:
knn_accuracy

0.9243976737745777

## Decision Trees with Numerical Features

Modify the implementation of decision trees to account for numerical input features.

In [15]:
# compute H(S)
def entropy(train_data, label, class_list):
    """
    Calculate the entropy of a dataset.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.

    Returns
    -------
    total_entr : float
        The entropy of the dataset.
    """
    # Get the total number of instances in the dataset
    total_row = train_data.shape[0]
    # Initialize the total entropy variable
    total_entr = 0

    # Iterate through each possible class in the label
    for c in class_list:
        # Count the number of points belonging to the current class
        total_class_count = train_data[train_data[label] == c].shape[0]

        # Check if there are instances of the class to avoid numerical errors
        if total_class_count > 0:
            # Calculate the entropy of the current class
            total_class_entr = - (total_class_count / total_row) * np.log2(total_class_count / total_row)
            # Add the entropy of the current class to the total entropy of the dataset
            total_entr += total_class_entr

    # Return the calculated total entropy of the dataset
    return total_entr

In [16]:
# compute H(S_j)
def feature_entropy(left_data, right_data, label, class_list):
    """
    Calculate the conditional entropy of a dataset split by a specific feature.

    Parameters
    ----------
    left_data : DataFrame
        Subset of the dataset where the feature has a specific value.
    right_data : DataFrame
        Subset of the dataset where the feature has another value.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.

    Returns
    -------
    ent : float
        The conditional entropy of the dataset split by the feature.
    """
    # Get the total number of points considered after the split
    row_count = left_data.shape[0] + right_data.shape[0]

    # Calculate the probabilities of the left and right subsets
    p_left = left_data.shape[0] / row_count
    p_right = right_data.shape[0] / row_count

    # Calculate the conditional entropy using the weighted average of entropies for left and right subsets
    ent = p_left * entropy(left_data, label, class_list) + p_right * entropy(right_data, label, class_list)

    # Return the calculated conditional entropy
    return ent

In [17]:
def split(feature_column, threshold):
    """
    Split the indices of data points based on a feature and a threshold.

    Parameters
    ----------
    feature_column : array-like
        The values of the feature for each data point.
    threshold : float
        The threshold value for splitting the data points.

    Returns
    -------
    left_rows : array-like
        Indices of data points where the feature value is less than or equal to the threshold.
    right_rows : array-like
        Indices of data points where the feature value is greater than the threshold.
    """
    # Find the indices of data points where the feature value is less than or equal to the threshold
    left_rows = np.argwhere(feature_column <= threshold).flatten()
    # Find the indices of data points where the feature value is greater than the threshold
    right_rows = np.argwhere(feature_column > threshold).flatten()

    # Return the indices for left and right subsets
    return left_rows, right_rows

In [18]:
def information_gain(data, feature_name, label, class_list, threshold):
    """
    Calculate the information gain after splitting the dataset based on a feature and a threshold.

    Parameters
    ----------
    data : DataFrame
        The dataset.
    feature_name : str
        The name of the feature for which information gain is calculated.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.
    threshold : float
        The threshold value for splitting the dataset.

    Returns
    -------
    feat_information_gain : float
        The information gain achieved by splitting the dataset based on the specified feature and threshold.
    """
    # Split the dataset into left and right subsets based on the feature and threshold
    left_rows, right_rows = split(data[feature_name].values, threshold)

    # Check if either subset is empty; if so, information gain is zero
    if len(left_rows) == 0 or len(right_rows) == 0:
        return 0

    # Calculate the entropy of the split dataset
    feat_entropy = feature_entropy(data.iloc[left_rows], data.iloc[right_rows], label, class_list)

    return feat_entropy


In [19]:
def get_split_thresholds(feature_column, n_thresholds):
    """
    Generate candidate split thresholds for a given feature column.

    Parameters
    ----------
    feature_column : array-like
        The values of the feature for each data point.
    n_thresholds : int
        The number of thresholds to generate.

    Returns
    -------
    thresholds : list of float
        List of candidate split thresholds for the feature column.
    """
    # Extract the values of the feature column
    feature_column = feature_column.values
    # Get the total number of data points
    n_data = len(feature_column)

    # Sort the feature column in ascending order
    sorted_column = np.sort(feature_column)

    # Check if there is more than one data point
    if len(feature_column) > 1:
        # Split the sorted feature column into n_thresholds + 1 partitions
        partitioned_array = np.array_split(sorted_column, n_thresholds + 1)

        # Calculate the midpoint between consecutive partitions as candidate thresholds
        thresholds = [(partitioned_array[i][-1] + partitioned_array[i + 1][0]) / 2 for i in range(len(partitioned_array) - 1)]
    else:
        # If there is only one data point, use it as the threshold
        thresholds = [feature_column[0]]

    # Return the list of candidate split thresholds
    return thresholds


In [20]:
def most_informative_feature(train_data, label, class_list, n_thresholds):
    """
    Find the most informative feature and its corresponding threshold for splitting the dataset.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.
    n_thresholds : int
        The number of thresholds to generate for each feature.

    Returns
    -------
    min_entropy_feature : str
        The name of the most informative feature.
    min_entropy_threshold : float
        The corresponding threshold for splitting the dataset based on the most informative feature.
    """
    # Get the list of features excluding the label
    feature_list = train_data.columns.drop(label)

    # Initialize variables to store the minimum entropy and corresponding feature and threshold
    min_entropy = float('inf')
    min_entropy_feature = None
    min_entropy_threshold = None

    # Iterate over each feature in the feature list
    for feature in feature_list:
        # Generate candidate split thresholds for the current feature
        thresholds = get_split_thresholds(train_data[feature], n_thresholds)

        # Iterate over each threshold
        for t in thresholds:
            # Calculate information gain for the current feature and threshold
            info_gain = information_gain(train_data, feature, label, class_list, t)

            # Check if the calculated information gain is less than the current minimum entropy
            if info_gain < min_entropy:
                # Update the minimum entropy and corresponding feature and threshold
                min_entropy = info_gain
                min_entropy_feature = feature
                min_entropy_threshold = t

    # Return the most informative feature and its corresponding threshold
    return min_entropy_feature, min_entropy_threshold

In [21]:
def is_leaf(train_data, label):
    """
    Check if a node in a decision tree is a leaf node.

    Parameters
    ----------
    train_data : DataFrame
        The dataset associated with the current node.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    bool
        True if the node is a leaf node (contains only one class), False otherwise.
    """
    # Get the unique classes in the current node
    classes_in_node = np.unique(train_data[label])

    # Check if there is only one class in the node
    if len(classes_in_node) == 1:
        # If there is only one class, the node is a leaf node
        return True
    else:
        # If there is more than one class, the node is not a leaf node
        return False

In [22]:
def leaf_class(train_data, label):
    """
    Determine the class of a leaf node in a decision tree.

    Parameters
    ----------
    train_data : DataFrame
        The dataset associated with the leaf node.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    leaf_class : str
        The class label assigned to the leaf node.
    """
    # Get the unique classes and their counts in the current leaf node
    class_list, count_class = np.unique(train_data[label], return_counts=True)

    # Find the index of the class with the highest count (most frequent class)
    idx = count_class.argmax()

    # Return the class label associated with the most frequent class in the leaf node
    return class_list[idx]

In [23]:
def make_tree(train_data, label, class_list, n_thresholds, cur_depth, min_samples, max_depth):
    """
    Recursively build a decision tree.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset associated with the current node.
    label : str
        The name of the column representing the class labels.
    class_list : list of str
        List of possible values of the class labels.
    n_thresholds : int
        The number of thresholds to generate for each feature.
    cur_depth : int
        The current depth of the decision tree.
    min_samples : int
        The minimum number of samples required to split a node.
    max_depth : int
        The maximum depth of the decision tree.

    Returns
    -------
    tree : dict or str
        The constructed decision tree represented as a nested dictionary. If a leaf node, returns the class label.
    """
    # Check stopping conditions for creating a leaf node
    if is_leaf(train_data, label) or cur_depth >= max_depth or len(train_data) <= min_samples:
        return leaf_class(train_data, label)
    else:
        # Increment the current depth for the next level of recursion
        cur_depth += 1

        # Find the most informative feature and its corresponding threshold for splitting
        split_feature, split_threshold = most_informative_feature(train_data, label, class_list, n_thresholds)

        # Split the dataset into left and right subsets based on the feature and threshold
        left_rows, right_rows = split(train_data[split_feature].values, split_threshold)

        # Check if either subset is empty; if so, create a leaf node
        if len(left_rows) == 0 or len(right_rows) == 0:
            return leaf_class(train_data, label)
        else:
            # Build the subtree
            split_condition = "{} <= {}".format(split_feature, split_threshold)
            sub_tree = {split_condition: []}

            # Recursive calls for the left and right branches
            left_branch = make_tree(train_data.iloc[left_rows], label, class_list, n_thresholds, cur_depth, min_samples, max_depth)
            right_branch = make_tree(train_data.iloc[right_rows], label, class_list, n_thresholds, cur_depth, min_samples, max_depth)

            # Check if both branches result in the same leaf class; if so, make the subtree a leaf
            if left_branch == right_branch:
                sub_tree = left_branch
            else:
                # Grow the tree by adding left and right branches to the split condition
                sub_tree[split_condition].append(left_branch)
                sub_tree[split_condition].append(right_branch)

            return sub_tree

In [24]:
def id3(train_data_m, label, n_thresholds=1, min_samples=4, max_depth=6):
    """
    Build a decision tree using the ID3 algorithm.

    Parameters
    ----------
    train_data_m : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.
    n_thresholds : int, optional
        The number of thresholds to generate for each feature.
    min_samples : int, optional
        The minimum number of samples required to split a node.
    max_depth : int, optional
        The maximum depth of the decision tree.

    Returns
    -------
    tree : dict or str
        The constructed decision tree represented as a nested dictionary. If a leaf node, returns the class label.
    """
    # Create a copy of the training dataset
    train_data = train_data_m.copy()

    # Get the unique classes of the label
    class_list = train_data[label].unique()

    # Start the recursion by calling the make_tree function
    tree = make_tree(train_data, label, class_list, n_thresholds, 0, min_samples, max_depth)

    # Return the constructed decision tree
    return tree

In [25]:
t = id3(train_data, 'Class')
print(t)

{'MajorAxisLength <= -0.26850534253244707': [{'MinorAxisLength <= -0.48101943425512017': [{'Area <= -0.7284149650882628': ['DERMASON', {'Perimeter <= -0.7436366901340172': ['DERMASON', {'roundness <= 0.34196234239628087': [{'ShapeFactor1 <= 0.742560155983303': ['SIRA', 'DERMASON']}, 'DERMASON']}]}]}, {'AspectRation <= -1.0816890411884459': ['SEKER', {'ShapeFactor2 <= 0.30636007444314095': ['SIRA', {'ShapeFactor1 <= 0.14890193826388914': [{'Compactness <= 0.8530383347908611': ['SIRA', 'SEKER']}, 'DERMASON']}]}]}]}, {'ShapeFactor1 <= -0.4188395892843162': [{'ShapeFactor2 <= -0.970259723740702': [{'Perimeter <= 1.2423181848501317': ['CALI', {'ShapeFactor1 <= -2.601011987728354': ['BOMBAY', 'CALI']}]}, {'roundness <= -0.655896778746935': ['BARBUNYA', {'Compactness <= -0.19114716081240862': ['CALI', {'roundness <= -0.07128514945601658': ['BARBUNYA', 'CALI']}]}]}]}, {'Compactness <= -1.1840258480155355': ['HOROZ', {'roundness <= 0.019868264666345184': [{'roundness <= -0.33450228559615847': [

In [26]:
def predict(test_point, tree):
    """
    Predict the class label for a given test point using a decision tree.

    Parameters
    ----------
    test_point : Series
        The test point for which the class label is predicted.
    tree : dict or str
        The decision tree used for prediction.

    Returns
    -------
    prediction : str
        The predicted class label for the test point.
    """
    # Base case: if the tree is a leaf node (a class label)
    if not isinstance(tree, dict):
        return tree

    # Recursive case: traverse the tree based on feature values
    question = list(tree.keys())[0]
    attribute, value = question.split(" <= ")

    # Check the condition and follow the appropriate branch
    if test_point[attribute] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # Recursive call on the selected branch
    return predict(test_point, answer)


def evaluate(tree, test_data, label):
    """
    Evaluate the accuracy of a decision tree on a test dataset.

    Parameters
    ----------
    tree : dict or str
        The decision tree to be evaluated.
    test_data : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    accuracy : float
        The accuracy of the decision tree on the test dataset.
    """
    correct_predict = 0
    wrong_predict = 0

    # Iterate over each row in the test dataset
    for index in tqdm(range(len(test_data.index))):
        # Predict the class label for the current test point
        result = predict(test_data.iloc[index], tree)

        # Check if the predicted value matches the expected value
        if result == test_data[label].iloc[index]:
            correct_predict += 1  # Increase correct count
        else:
            wrong_predict += 1  # Increase incorrect count

    # Calculate and return the accuracy
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy

In [27]:
evaluate(t, test_data, 'Class')

100%|██████████| 3611/3611 [00:00<00:00, 11327.67it/s]


0.8773193021323733

## Gaussian Naive Bayes
Modify the implementation of naive Bayes to accout for numerical input features. The likelihood of each class ($p(data|class)$) is assumed to be a Gaussian $\frac{1}{\sqrt(\sigma^2 2 \pi)} \exp (-\frac{1}{2} \frac{(x-\mu)^2}{\sigma^2})$, where $\mu, \sigma^2$ are the mean and the variance for each class;

In [28]:
from typing import Dict, Tuple

def prior(train_data: pd.DataFrame, label: str) -> np.ndarray:
    """
    Calculate the log prior probabilities for each class in the dataset.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    priors : array-like
        The log prior probabilities for each class.
    """
    # Calculate the prior probabilities for each class
    priors = train_data.groupby(by=label).apply(lambda x: len(x) / len(train_data))

    # Return the log of the prior probabilities as an array
    return np.log(priors).values


def mean_variance(train_data: pd.DataFrame, label:str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate the mean and variance for each feature in the dataset, grouped by class.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    mean : array-like
        The mean values for each feature and class.
    variance : array-like
        The variance values for each feature and class.
    """
    # Calculate the mean values for each feature and class
    mean = train_data.groupby(by=label).apply(lambda x: x.mean(axis=0))

    # Calculate the variance values for each feature and class
    variance = train_data.groupby(by=label).apply(lambda x: x.var(axis=0))

    # Return the mean and variance as arrays
    return (mean.values, variance.values + 1e-9)


def gaussian_density(mean: np.ndarray, variance: np.ndarray, point: np.ndarray) -> np.ndarray:
    """
    Calculate the Gaussian probability density for a given point.

    Parameters
    ----------
    mean : array-like
        The mean values for each feature and class.
    variance : array-like
        The variance values for each feature and class.
    point : array-like
        The values of the features for a given point.

    Returns
    -------
    density : array-like
        The Gaussian probability density for the given point.
    """
    # Calculate the Gaussian probability density for each feature
    d = (1 / np.sqrt(2*np.pi*variance)) * np.exp((-(point - mean)**2) / (2*variance))

    # Return the density as an array
    return d


def train_gaussian_naive_bayes(train_data: pd.DataFrame, label: str) -> Dict[str, np.ndarray]:
    """
    Train a Gaussian Naive Bayes classifier.

    Parameters
    ----------
    train_data : DataFrame
        The training dataset.
    label : str
        The name of the column representing the class labels.

    Returns
    -------
    model : dict
        A dictionary containing the parameters of the trained Gaussian Naive Bayes model.
    """
    # Calculate the mean and variance for each feature and class
    mean, variance = mean_variance(train_data, label)

    # Calculate the log prior probabilities for each class
    priors = prior(train_data, label)

    # Get unique class labels and their count
    unique_labels = sorted(train_data[label].unique())
    n_labels = len(unique_labels)

    # Construct and return the Gaussian Naive Bayes model
    return {'n_labels': n_labels, 'unique_labels': unique_labels, 'n_classes': n_labels, 'mean': mean,
            'variance': variance, 'prior': priors}


In [29]:
gaus_bayes = train_gaussian_naive_bayes(train_data, 'Class')

In [30]:
gaus_bayes['unique_labels']

['BARBUNYA', 'BOMBAY', 'CALI', 'DERMASON', 'HOROZ', 'SEKER', 'SIRA']

In [31]:
np.exp(gaus_bayes['prior'])

array([0.0967, 0.0381, 0.1176, 0.2638, 0.1421, 0.147 , 0.1947])

In [32]:
print(gaus_bayes['mean'].shape)

(7, 16)


In [33]:
def likelihood(point: np.ndarray, mean: np.ndarray, variance: np.ndarray, n_classes: int, n_feat: int) -> np.ndarray:
    """
    Calculate the log posterior probabilities for each class given a data point.

    Parameters
    ----------
    point : array-like
        The values of the features for a given data point.
    mean : array-like
        The mean values for each feature and class.
    variance : array-like
        The variance values for each feature and class.
    class_list : array-like
        The unique class labels.
    n_classes : int
        The number of classes.
    n_feat : int
        The number of features.

    Returns
    -------
    posteriors : array-like
        The log posterior probabilities for each class.
    """
    likelihoods = []
    for i in range(n_classes):
        likelihood = 0
        for j in range(n_feat):
            likelihood += np.log(gaussian_density(mean[i][j], variance[i][j], point[j]))
        likelihoods.append(likelihood)
    return np.array(likelihoods)


def predict(test_data: pd.DataFrame, gaus_bayes:Dict[str, np.ndarray]) -> np.ndarray:
    """
    Predict the class labels for a given test dataset using a trained Gaussian Naive Bayes model.

    Parameters
    ----------
    test_data : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.
    gaus_bayes : dict
        A dictionary containing the parameters of the trained Gaussian Naive Bayes model.

    Returns
    -------
    predictions : array-like
        The predicted class labels for the test dataset.
    """
    predictions = []
    n_feat = len(test_data.columns) - 1
    for i in range(len(test_data)):
        pr = gaus_bayes['prior']
        like = likelihood(test_data.iloc[i, :-1], gaus_bayes['mean'], gaus_bayes['variance'],
                          gaus_bayes['n_classes'], n_feat)
        prob = pr + like
        max_prob_class_idx = np.argmax(prob)
        predictions.append(gaus_bayes['unique_labels'][max_prob_class_idx])
    return np.array(predictions)


def evaluate(test_data: pd.DataFrame, label: str, gaus_bayes: Dict[str, np.ndarray]) -> float:
    """
    Evaluate the accuracy of a Gaussian Naive Bayes model on a test dataset.

    Parameters
    ----------
    test_data : DataFrame
        The test dataset.
    label : str
        The name of the column representing the class labels.
    gaus_bayes : dict
        A dictionary containing the parameters of the trained Gaussian Naive Bayes model.

    Returns
    -------
    accuracy : float
        The accuracy of the Gaussian Naive Bayes model on the test dataset.
    """
    gaus_pred = predict(test_data, gaus_bayes)
    correct_predict = 0
    wrong_predict = 0
    for index in tqdm(range(len(test_data.index))):
        if gaus_pred[index] == test_data[label].iloc[index]:
            correct_predict += 1
        else:
            wrong_predict += 1
    accuracy = correct_predict / (correct_predict + wrong_predict)
    return accuracy

In [34]:
evaluate(test_data, 'Class', gaus_bayes)

/Users/nicolas/Desktop/Phd/Intro_to_ML_25-26/venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
100%|██████████| 3611/3611 [00:00<00:00, 163490.88it/s]


0.903073940736638

In [35]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

cols = list(set(train_data.columns) - {'Class'})

nb.fit(train_data[cols], train_data['Class'])

pred = nb.predict(test_data[cols])

In [36]:
print((pred == label_test.values).sum()/len(pred))

0.903073940736638
